In [ ]:
# 📓 04_extract_diameters.ipynb

import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import os
import pandas as pd
from src.mesh_utils import load_mesh
from src.io import load_point_ids

# Load point IDs
id_list = load_point_ids('output/pa_point_ids.json')
subject_dir = 'data/subject_meshes/'
subject_files = sorted(os.listdir(subject_dir))

records = []

for fname in subject_files:
    mesh = load_mesh(os.path.join(subject_dir, fname))
    points = mesh.points[id_list]

    # Center the points
    points_centered = points - np.mean(points, axis=0)

    # Apply PCA to find major and minor axes
    pca = PCA(n_components=2)
    pca.fit(points_centered)
    
    # Project points onto principal components
    proj = pca.transform(points_centered)
    diameter_major = np.max(proj[:, 0]) - np.min(proj[:, 0])
    diameter_minor = np.max(proj[:, 1]) - np.min(proj[:, 1])

    records.append({
        'subject': fname,
        'major_diameter': diameter_major,
        'minor_diameter': diameter_minor
    })

# Save result
df = pd.DataFrame(records)
df.to_csv('output/subject_diameters.csv', index=False)
df.head()